# 🤖 AI 기반 머신러닝 서울 맛집 추천 시스템
이 프로젝트는 sample 리뷰 데이터를 불러와 전처리하고,
지하철역, 음식종류, 예산, 연령층 등 점수를 계산하여 추천하는 과정을 담습니다.

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import gradio as gr
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# ------------------------
# 1. 데이터 로드 및 모델 학습
# ------------------------
df = pd.read_excel('/content/drive/MyDrive/Restaurants.xlsx')
df['가격대'] = df['가격대'].astype(float)

# Synthetic label 생성
ideal_budget = 12000
df['budget_diff'] = abs(df['가격대'] - ideal_budget)

df['label'] = np.where(
   (df['평점'] >= 4.0) & (df['budget_diff'] <= 3000),
      1,
      0
)

df_ml = pd.get_dummies(df, columns=['음식종류', '연령층', '방문목적'])

X = df_ml.drop(columns=['식당명', '리뷰', '위치(지하철역)', 'label'])
y = df_ml['label']

X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42 
)
model = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.05)
model.fit(X_train, y_train)

df['predict_score'] = model.predict_proba(X)[:, 1]

# ------------------------
# 2. 추천 함수
# ------------------------
def recommend_ai(region, food_type, budget, age):

    # ----- 기본 필터 -----
    filtered = df[(df['위치(지하철역)'] == region) &
                  (df['음식종류'] == food_type)]

    if filtered.empty:
        return '조건에 맞는 식당이 없습니다.'

    filtered = filtered.copy()

    # ----- 🔥 예산 필터: budget - 3000 ~ budget + 3000 -----
    min_price = budget - 3000
    max_price = budget + 3000

    filtered = filtered[(filtered['가격대'] >= min_price) 
                        (filtered['가격대'] <= max_price)]

    if filtered.empty:
        return f'예산 범위({min_price}원 ~ {max_price}원)에 맞는 식당이 없습니다.'

    # 예산 차이 계산
    filtered['user_budget_diff'] = abs(filtered['가격대'] - budget)
    # 최종 점수 (ML 70% + 가격 적합도 30%)
    filtered['final_score'] = ( 
        filtered['predict_score'] * 0.7 +
        (1 / (filtered['user_budget_diff'] + 1)) * 0.3 
    )

    # Top 5
    result = filtered.sort_values('final_score', ascending=False).head(5)
    top = result.iloc[0]

    # -----------------------------
    # 🏆 TOP 1
    # -----------------------------











































# ------------------------
# CSV 불러오기
df = pd.read_csv('../data/sample_reviews.csv')
df.head()